In [ ]:
from pyspark.sql.types import BinaryType, StringType, ArrayType
from pyspark.sql.functions import udf, row_number, monotonically_increasing_id, col, collect_list, concat_ws, explode
from pyspark.sql import SparkSession, Window

from protocol_buffers import document_pb2

import pickle

In [ ]:
spark_drive_gbs = 3
spark_executor_gbs = 50
cores = 10

para_path = '/nfs/trec_news_track/index/2018_bm25_rm3_chunks_full_v1/'


In [ ]:
print('\n//////// RUNNING WITH CORES {} //////////'.format(cores))
spark = SparkSession.\
    builder\
    .appName('test')\
    .master('local[{}]'.format(cores)) \
    .config("spark.driver.memory", '{}g'.format(spark_drive_gbs)) \
    .config("spark.executor.memory", '{}g'.format(spark_executor_gbs)) \
    .config("spark.driver.maxResultSize", '{}g'.format(spark_drive_gbs)) \
    .getOrCreate()

In [ ]:
df = spark.read.parquet(para_path)
df.printSchema()